# Date Mining Assignment

In [ ]:
import numpy as np
import pandas as pd
from IPython.display import display, Markdown, Latex
pd.set_option('max_colwidth', 1000)

import re, os
for d in ["src","data"]: os.makedirs(d, exist_ok=True)

## Load Dataset

In [ ]:
URL = "https://kmurphy.bitbucket.io/modules/Data_Mining_2/topics/08-Text_Mining/20-Mining_Dates/files/"

for filename, dest in [("public.csv", "src")]:
    
    source = f"{URL}/{filename}"
    target = f"{dest}/{filename}"

    if not os.path.isfile(target):
        print (f"Downloading remote file {filename} to folder {dest}", sep="")
        import urllib.request
        urllib.request.urlretrieve(source, target)
    else:
        print(f"Using local copy of {filename} in folder {dest}")

In [ ]:
df = pd.read_csv("src/public.csv")
df.head()

## Outline of Solution Pipeline

 * First perform a cleaning step. This greatly reduces the number of regular expressions needed.
     - you to figure out specifics here
   
 * Define two helper functions to simply application of regular expression and to display which parts of dataset are matched / not matched.
 * Build a sequence of regular expressions (starting from most restrictive to avoid false positives) to apply to dataset.
 * At each match/iteration use examples of unmatched rows to determine next possible regular expression.

---

## Cleaning Step

As a result of this step, the dataframe will have columns

 * __Code__ unchanged
 * __Raw__ unchanged
 * __Data__ cleaned version of __Raw__
 * __Iter__ number of regex that matched this row (for development purposes)
 * __Match__ regex object result for this row (for development purposes, you might not use)
 * __Day__ day (or zero if not set)
 * __Month__ month (or zero if not set)
 * __Year__ year (or zero if not set)

In [ ]:
def clean_message(src):
    
    # you TODO
    s = src
    
    return s

df['Data'] = df.Raw.map(clean_message)
df["Iter"] = 0        
df["Match"] = 0     
df["Day"] = 0
df["Month"] = 0
df["Year"] = 0

## Helper Functions

In [ ]:
def info(n=None, unmatched=True, head=5):
    "Helper function to display results of a match and some rows not matched so far."
    
    n = df.Match.max() if n is None else n
    for nn in ([n,0] if n!=0 and unmatched else [n]):
        display(Markdown("**Numer of rows with iter=%s: %s**" % (nn,len(df[df.Iter==nn]))))
        display(df.loc[df.Iter==nn,["Code","Data","Iter","Match", "Day","Month","Year"]].head(head))
info()

In [ ]:
def verify_dates(df):
    # you TODO (or skip if you don't need it)
    return df

----
## Sequence of Regex's 

Feel free to change what I am doing here. The structure is simple, but adaptable:
 * Construct the regex
     * Pick a row and develop/test against it
 * Apply regex 
     * Set counter __iter__
     * apply to unmatched rows in __df__ to create a second dataframe, __df2__, with columns __Day__,__Month__,__Year__
     * Drop unmatch rows --- identified by NaN.
 * Test matched rows (I'm not giving you this, and you might not need it anyway, depending on your regex)
     * It is possible that the regex matched something that was not a valid date -- you want to remove these rows from __df2__. 
 * Update matched rows in __df__.
     * Save appropriate entry in __Day__, __Month__, __Year__, and __iter__.
     
---
---

### 1 - mm/dd/yyyy 

In [ ]:
# Construct the regex
tmp = df.loc[df.Code==10430271,"Data"]
display(tmp)

regex = r"([\d]{1,2})/([\d]{1,2})/(\d{2,4})"
#tmp.str.extract(regex)

In [ ]:
# apply regex

iter = 1

df2 = df[df.Iter==0].Data.str.extract(regex)
df2.dropna(inplace=True)
display(Markdown(" * Number of row matched = %s" % df2.shape[0]))
display(df2.head())

In [ ]:
# test result
df2 = verify_dates(df2)

In [ ]:
# save result

df.loc[(df.Iter==0) & (df2.Year), "Day"] = df2["Day"]
df.loc[(df.Iter==0) & (df2.Year), "Month"] = df2["Month"]
df.loc[(df.Iter==0) & (df2.Year), "Year"] = df2["Year"]

df.loc[(df.Iter==0) & (df2.Year),"Iter"] = iter
info(iter)